In [759]:
# import datetime
import math
import numpy as np
import scipy as sp
import scipy.fftpack
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.image as image
from matplotlib.offsetbox import  OffsetImage

#matplotlib inline
import pyproj
import pymap3d as pm

from astropy.time import Time

In [760]:
stacode='alex'
## input data
data = pd.read_csv("data/turkeq/{}_kin03700_23.cts".format(stacode), sep=" ")
data['Lat'], data['Lon'], data['Alt'] = pm.ecef2geodetic(data['X'], data['Y'], data['Z'], ell=None, deg=True)

data['mjdTim'] = data['mjd']+data['sod']/(3600*24)
data['Date'] = Time(data['mjdTim'], format='mjd').datetime

data = data.set_index(['Date'])
header= ['X','Y','Z',]
data
data.to_csv('{}_coor.csv'.format(stacode), columns=header)

In [761]:
# split data
df1 = data.loc['2023-02-06 01:10:00' : '2023-02-06 01:40:00', :]

df2 = data.loc['2023-02-06 10:20:00' : '2023-02-06 10:50:00', :]

In [762]:
#azimuth
from pyproj import Geod
#earthquake 1
lat_eq1 = 37.226
lon_eq1 = 37.014

lat1 = df1['Lat'][0]
lon1 = df1['Lon'][0]

g = Geod(ellps='WGS84') 
a1 = g.inv(lon1, lat1, lon_eq1, lat_eq1, radians=0)[0]
a1=math.radians(a1)
   
if a1<0:
        a1=abs(a1)
        a1=2*np.pi-a1

#earthquake 2
lat_eq2 = 38.011
lon_eq2 = 37.196

lat2 = df2['Lat'][0]
lon2 = df2['Lon'][0]

a2 = g.inv(lon2, lat2, lon_eq2, lat_eq2, radians=0)[0]
a2=math.radians(a2)

if a2<0:
        a2=abs(a2)
        a2=2*np.pi-a2
        


In [763]:
#geodesic distance from epicenter
from geopy.distance import geodesic

epicenter1 = (lat_eq1,lon_eq1)
epicenter2 = (lat_eq2, lon_eq2)

station_coor = (data['Lat'][0], data['Lon'][0])
dist1 = (geodesic(epicenter1, station_coor).km)
dist2 = (geodesic(epicenter2, station_coor).km)

#print(dist1)
#print(dist2)

In [764]:
line = '{} '.format(stacode)
#for c in ['X', 'Y', 'Z', 'Lat', 'Lon']:
   # mean_c = data[c].mean()
   # line += '{:}'.format(mean_c)
line += '{:} {:} {:} {:} {:} {:}'.format(lat1, lon1, a1*180/math.pi, a2*180/math.pi, dist1, dist2)
#line contains station code, station lat and lon, azimuth for 1st eq, azimuth for 2nd eq, distance in km from 1st epicenter, distance in km from 2nd epicenter 
print(line)

with open("Outputs/output_azimuth_dist.txt", "a") as fout:
    fout.write("{:} \n".format(line))
fout.close()


dyng 38.07855205914097 23.9324354433117 86.26892806543607 1163.296424642552
